In [214]:
import pandas as pd
import math as math
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.special import boxcox, inv_boxcox

import numpy as np
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit


In [275]:
data = pd.read_csv(r"C:\Users\patte\Documents\apps\covidLSTM\COVID-19\csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_confirmed_global.csv");

In [316]:
uk_data = data.loc[data['Country/Region'] == '']
uk_data

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20


In [328]:
def fullNaN(countries_data):
    result = []
    for country_data in countries_data:
        result.append(country_data.fillna('Total'))
    return result

def country_with_province_data_prep(countries_data):
    result = []
    for country_data in countries_data:
        country_name = country_data['Country/Region'].values[0]
        del country_data['Country/Region']
        del country_data['Lat'];
        del country_data['Long'];
        country_data = country_data.melt(id_vars=['Province/State'], var_name='Date', value_name='Confirmed Cases');
        country_data['Date'] = pd.to_datetime(country_data['Date']);
        country_data = country_data.pivot(index='Date', columns='Province/State', values='Confirmed Cases');
        country_total = pd.DataFrame()
        country_total[country_name + ' Total Confirmed Cases '] = country_data.sum(axis=1)
        result.append(country_total)
    return result

def get_training_data_no_provinces():
    korea_data = data.loc[data['Country/Region'] == 'Korea, South']
    italy_data = data.loc[data['Country/Region'] == 'Italy']
    spain_data = data.loc[data['Country/Region'] == 'Spain']
    germany_data = data.loc[data['Country/Region'] == 'Germany']

    return [korea_data, italy_data, spain_data, germany_data]

def get_training_data_with_provinces():
    france_data = data.loc[data['Country/Region'] == 'France']
    uk_data = data.loc[data['Country/Region'] == 'United Kingdom']
    china_data = data.loc[data['Country/Region'] == 'China']
    
    return [france_data, uk_data, china_data]

def country_no_province_data_prep(countries):
    result = []
    for country_data in countries:
        country_name = country_data['Country/Region'].values[0]
        del country_data['Lat']
        del country_data['Long']
        del country_data['Province/State']
        country_data = country_data.melt(id_vars=['Country/Region'], var_name='Date', value_name='Confirmed Cases')
        ## Data prep for Korea
        country_data['Date'] = pd.to_datetime(country_data['Date']);
        country_data = country_data.pivot(index='Date', columns='Country/Region', values='Confirmed Cases');
        country_total = pd.DataFrame()
        country_total[country_name + ' Total Confirmed Cases'] = country_data.sum(axis=1)
        result.append(country_total)
    return result

In [329]:
countries_provinces_data = get_training_data_with_provinces()
countries_data = get_training_data_no_provinces()
x1 = country_with_province_data_prep(countries_provinces_data)
x2 = country_no_province_data_prep(countries_data)
input_data = x1 + x2

In [330]:
visualize = pd.concat(input_data, axis=1)
visualize.iloc[1:20]

,France Total Confirmed Cases,United Kingdom Total Confirmed Cases,China Total Confirmed Cases,"Korea, South Total Confirmed Cases",Italy Total Confirmed Cases,Spain Total Confirmed Cases,Germany Total Confirmed Cases
Date,,,,,,,
2020-01-23,0,0,643,1,0,0,0
2020-01-24,2,0,920,2,0,0,0
2020-01-25,3,0,1406,2,0,0,0
2020-01-26,3,0,2075,3,0,0,0
2020-01-27,3,0,2877,4,0,0,1
2020-01-28,4,0,5509,4,0,0,4
2020-01-29,5,0,6087,4,0,0,4
2020-01-30,5,0,8141,4,0,0,4
2020-01-31,5,2,9802,11,2,0,5


In [8]:
## Standardization
def normalizeData(data):
    for data in datas
        scaler = MinMaxScaler().fit(data)
        data = scaler.transform(data)
        return data, scaler

def split_sequence(sequence, n_steps):
    len_seq = sequence.shape[1]
    X, y = list(), list()
    for i in range(len_seq):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len_seq-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[:, i:end_ix], sequence[:, end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return asarray(X), asarray(y)

In [11]:
# korea_total, korea_scaler = normalizeData(korea_total)
# china_total, china_scaler = normalizeData(china_total)

confirmed_cases_array = np.array([asarray(korea_total), asarray(china_total)])

In [18]:
n_steps = 1
X, y = split_sequence(confirmed_cases_array, n_steps)

In [22]:
confirmed_cases_array.shape

(2, 58, 1)

In [23]:
X.shape

(57, 2, 1, 1)

In [26]:
## Splitting into validation and test sets
number_of_rows = X.shape[0]
n_train = math.ceil(number_of_rows*80/100)
n_test = n_train + math.floor(number_of_rows*10/100)
X_train, y_train, X_val, y_val, X_test, y_test = X[:n_train], y[:n_train], X[n_train:n_test], y[n_train:n_test], X[n_test:], y[n_test:]